<a href="https://colab.research.google.com/github/alexcpn/tranformer_learn/blob/main/bloom_560m_overfitting_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.28.1
!pip install accelerate

In [16]:
#upload files to your colab environment
!wget https://raw.githubusercontent.com/alexcpn/tranformer_learn/main/data/small_3.txt
#!wget https://gist.githubusercontent.com/alexcpn/54e88130f9d186494f1c3ce5e83263b4/raw/7cdf5f93b819024c58a891fc808fbdbe052d0eb1/small_3_mixed.txt

--2023-06-19 10:07:26--  https://raw.githubusercontent.com/alexcpn/tranformer_learn/main/data/small_3.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56513 (55K) [text/plain]
Saving to: ‘small_3.txt.1’

small_3.txt.1       100%[===================>]  55.19K  --.-KB/s    in 0.002s  

2023-06-19 10:07:26 (35.2 MB/s) - ‘small_3.txt.1’ saved [56513/56513]



In [17]:
train_path = 'small_3.txt'


In [18]:
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import AutoTokenizer

def load_dataset(path,tokenizer):
    dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return dataset,data_collator

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
train_dataset,data_collator = load_dataset(train_path,tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [19]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("bigscience/bloom-560m")


training_args = TrainingArguments(
    output_dir="./bloom-560-small3-v1", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=250, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    #eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [20]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.741600
1000,0.128800
1500,0.047400
2000,0.029600
2500,0.018000
3000,0.013700
3500,0.007400
4000,0.005000
4500,0.004700


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1662 in train                    │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1929 in _inner_training_loop     │
│                                                                                                  │
│   1926 │   │   │   │   │   with model.no_sync():                                                 │
│   1927 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1928 │   │   │   │   else:                                                                     │
│ ❱ 1929 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1930 │   │   │   │                                                                             │
│   1931 │   │   │   │   if (                                                                      │
│   1932 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2717 in training_step            │
│                                                                                                  │
│   2714 │   │   │   # loss gets scaled under gradient_accumulation_steps in deepspeed             │
│   2715 │   │   │   loss = self.deepspeed.backward(loss)                                          │
│   2716 │   │   else:                                                                             │
│ ❱ 2717 │   │   │   loss.backward()                                                               │
│   2718 │   │                                                                                     │
│   2719 │   │   return loss.detach()                                                              │
│   2720                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/_tensor.py:487 in backward                         │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,                                                │
│    485 │   │   │   │   inputs=inputs,                                                            │
│    486 │   │   │   )                                                                             │
│ ❱  487 │   │   torch.autograd.backward(                                                          │
│    488 │   │   │   self, gradient, retain_graph, create_gra


Short detail about training -
- Step	Training Loss
- 500	2.263500
- 1000	0.205800
- 1500	0.029900

Took about 6 hours in Colab Free in TPU runtime

Stopped at Epoch 35 as the Training loss was pretty low

1655/2350 7:15:15 < 3:03:00, 0.06 it/s, Epoch 35.19/50]

In [21]:
trainer.save_model()

In [40]:
!zip -r bloom-560-small3-v1.zip bloom-560-small3-v1/config.json  bloom-560-small3-v1/training_args.bin  bloom-560-small3-v1/pytorch_model.bin bloom-560-small3-v1/generation_config.json


  adding: bloom-560-small3-v1/config.json (deflated 49%)
  adding: bloom-560-small3-v1/training_args.bin (deflated 48%)
  adding: bloom-560-small3-v1/pytorch_model.bin (deflated 7%)
  adding: bloom-560-small3-v1/generation_config.json (deflated 28%)


In [41]:
!cp bloom-560-small3-v1.zip ./drive/MyDrive/models

# Test Model

In [ ]:
#!cp ./drive/MyDrive/models/bloom-560-small3-v1.zip . #if you are taking the fine tuned model from drive

In [ ]:
#!unzip bloom-560-small3-v1.zip

In [22]:
from transformers import pipeline

test = pipeline('text-generation',model='./bloom-560-small3-v1/', tokenizer='bigscience/bloom-560m')

In [23]:
test('An alkaline medium favours', max_new_tokens=512)

[{'generated_text': 'An alkaline medium favours bacterial growth; and moisture is a necessary condition; spores, however, can survive the want of water for much longer periods than fully developed bacteria. The necessity for oxygen varies in different species. Those that require oxygen are known as  aërobic bacilli  or  aërobes ; those that cannot live in the presence of oxygen are spoken of as  anaërobes ; those that cannot live in the presence of oxygen are spoken of as  anaërobes. The necessity for oxygen varies in different species. Those that require oxygen are known as  anaërobes ; those that cannot live in the presence of oxygen are spoken of as  anaërobes. The necessity for oxygen varies in different species. Those that require oxygen are known as  anaërobes ; those that cannot live in the presence of oxygen are spoken of as  anaërobes. The necessity for oxygen varies in different species. Those that require oxygen are known as  anaërobes ; those that cannot live in the presenc

from the passage
[ An alkaline medium favours bacterial growth; and moisture is a necessary condition; spores, however, can survive the want of water for much longer periods than fully developed bacteria. The necessity for oxygen varies in different species. Those that require oxygen are known as  aërobic bacilli  or  aërobes ; those that cannot live in the presence of oxygen are spoken of as  anaërobes . The great majority of bacteria, however, while they prefer to have oxygen, are able to live without it, and are called  facultative anaërobes

In [25]:
test('Streptococci are met with in', max_new_tokens=120,num_return_sequences=1)

[{'generated_text': 'Streptococci are met with in the course of a few days. When they divide irregularly, and form grape-like bunches, they are known as  staphylococci, and to this variety the commonest pyogenic or pus-forming organisms belong (Fig. 5). When division takes place only in one axis, so that long chains are formed, the term  streptococcus  is applied (Fig. 2). Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped cells, more or less spiral or w'}]

The inital part is exaclty as in the passage [Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped bacteria, usually at least twice as long as they are broad (Fig. 4). Some multiply by fission, others by sporulation. Some forms are motile, others are non-motile.]

The last line

from passage [Tuberculosis, tetanus, anthrax, and many other surgical diseases are due to different forms of bacilli.  Spirilla  are long, slender, thread-like cells, more or less spiral or wavy.]

The last line generated is not correct

"Some forms are motile only in virtue of the contractility of the protoplasm, "

is there in another pace where "motile" is reffered

whereas the following is pure hallicunation

"some in virtue of the fibroblasts which they carry. Others are'}]"

In [28]:
test('Streptococci', max_new_tokens=100,num_return_sequences=1)

[{'generated_text': 'Streptococci, of which the bacteria of gonorrhœa and pneumonia are examples (Fig. 5). When they divide irregularly, and form grape-like bunches, they are known as  staphylococci, and to this variety the commonest pyogenic or pus-forming organisms belong (Fig. 2). When division takes place only in one axis, so that long chains are formed, the term  streptococcus  is applied (Fig. 3). Streptococci are met with'}]

In [29]:
test('Metchnikoff', max_new_tokens=100,num_return_sequences=1)

[{'generated_text': "Metchnikoff's disease, diabetes, syphilis, scurvy, or alcoholism, also impedes healing. Infection by disease-producing micro-organisms or  pathogenic bacteria  is, however, the most potent factor in disturbing the natural process of repair in wounds.\n\nSURGICAL BACTERIOLOGY The influence of micro-organisms in the causation of disease, and the rôle played by them in interfering with the natural process of repair, are so important that the science of applied"}]

In [30]:
test('To this process Metchnikoff', max_new_tokens=100,num_return_sequences=1)

[{'generated_text': 'To this process Metchnikoff gave the name of  phagocytosis, and he recognised two forms of  phagocytes : (1) the  microphages, which are the polymorpho-nuclear leucocytes of the blood; and (2) the  macrophages, which include the larger hyaline matrix. These fibroblasts are large irregular nucleated cells derived mainly from the proliferation of the fixed connective-tissue cells of the part, and to a less extent from the lymphocytes and other mon'}]

In [31]:
test('Phagocytosis', max_new_tokens=100,num_return_sequences=1)

[{'generated_text': 'Phagocytosis, and the rôle played by them in interfering with the natural process of repair. In this way a damaged part is repaired by ordinary cicatricial tissue derived from the proliferating cells of the perichondrium. The cells of the damaged tissues, under the influence of this irritation, undergo certain proliferative changes, which are designed to restore the normal structure and function of the tissue cells. These changes are common to the processes of repair and repair; no hard-and-fast line'}]

In [32]:
test('During the process of phagocytosis,', max_new_tokens=100,num_return_sequences=1)

[{'generated_text': 'During the process of phagocytosis, the polymorpho-nuclear leucocytes in the circulating blood increase greatly in numbers ( leucocytosis ), as well as in their phagocytic action, and in the course of destroying the bacteria they produce certain ferments which enter the blood serum. These are known as  opsonins  or  alexins, and they act on the bacteria by a process comparable to narcotisation, and render them an easy prey for the phagocytes.  Artificial or Passive Immunity. '}]

In [34]:
test(' diplococci ', max_new_tokens=100,num_return_sequences=1)

[{'generated_text': ' diplococci  of which the bacteria of gonorrhœa and pneumonia are examples (Fig. 5). When they divide irregularly, and form grape-like bunches, they are known as  staphylococci, and to this variety the commonest pyogenic or pus-forming organisms belong (Fig. 2). When division takes place only in one axis, so that long chains are formed, the term  streptococcus  is applied (Fig. 3). Streptococci are met with in'}]

In [35]:
 test('Cocci  or  micrococci', max_new_tokens=100,num_return_sequences=1)

[{'generated_text': 'Cocci  or  micrococci  are usually found in the blood. Some of the varieties, such as those of erysipelas, diphtheria, and acute osteomyelitis, vary considerably. There is evidence that a  mixed infection  that is, the introduction of more than one species of organism, for example, the tubercle bacillus and a pyogenic staphylococcus increases the severity of the resulting disease. If one of the varieties gain the ascendancy, the poisons produced by the others'}]

In [36]:
 test('Bacteria are most conveniently', max_new_tokens=100,num_return_sequences=1)

[{'generated_text': 'Bacteria are most conveniently classified according to their shape. Thus we recognise (1) those that are globular  cocci ; (2) those that resemble a rod  bacilli ; (3) the spiral or wavy forms  spirilla.  Cocci  or  micrococci  are minute round bodies, averaging about 1 µ in diameter. The great majority are non-motile. They multiply by fission; and when they divide in such a way that the resulting cells remain in pairs, are called '}]

In [39]:
 test('given the context "Thus we recognise (1) those that are globular  cocci ; (2) those that resemble a rod  bacilli ; (3) the spiral or wavy forms  spirilla .  Cocci  or  micrococci  are minute round bodies, averaging about 1 µ in diameter. The great majority are non-motile. They multiply by fission; and when they divide in such a way that the resulting cells remain in pairs, are called  diplococci , of which the bacteria of gonorrhœa and pneumonia are examples (Fig. 5). When they divide irregularly, and form grape-like bunches, they are known as  staphylococci , and to this variety the commonest pyogenic or pus-forming organisms belong' +
 'answer "What are Cocci  or  micrococci', max_new_tokens=100,num_return_sequences=1)

[{'generated_text': 'given the context "Thus we recognise (1) those that are globular  cocci ; (2) those that resemble a rod  bacilli ; (3) the spiral or wavy forms  spirilla .  Cocci  or  micrococci  are minute round bodies, averaging about 1 µ in diameter. The great majority are non-motile. They multiply by fission; and when they divide in such a way that the resulting cells remain in pairs, are called  diplococci , of which the bacteria of gonorrhœa and pneumonia are examples (Fig. 5). When they divide irregularly, and form grape-like bunches, they are known as  staphylococci , and to this variety the commonest pyogenic or pus-forming organisms belonganswer "What are Cocci  or  micrococci  are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped cells, averaging about 1 µ in diameter. The great majority are non-motile. They multiply by a screw-like contraction of the protoplasm, and when it is embedded in 